# Symptom Prediction using MLP Model

In [8]:

# Import required libraries
import pandas as pd
import torch
from torch import nn
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split


In [4]:
!pip install kaggle
!pip install torch
!pip install scikit-learn
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

import kagglehub

# Download latest version
path = kagglehub.dataset_download("itachi9604/disease-symptom-description-dataset")

print("Path to dataset files:", path)

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split,KFold,cross_val_score,GridSearchCV
from sklearn.svm import SVC
from sklearn.metrics import f1_score, accuracy_score, confusion_matrix,classification_report,confusion_matrix,precision_score,roc_curve
import seaborn as sns
from sklearn.utils import shuffle
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics
import warnings
import torch
import os

warnings.filterwarnings("ignore")

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

DATASET_PATH = path

class_dir = os.path.join(DATASET_PATH)
for filename in os.listdir(class_dir):
  print(filename)


df = pd.read_csv(os.path.join(DATASET_PATH, 'dataset.csv'))
df = shuffle(df,random_state=42)

for col in df.columns:
    df[col] = df[col].str.replace('_',' ')

null_checker = df.apply(lambda x: sum(x.isnull())).to_frame(name='count')

cols = df.columns
data = df[cols].values.flatten()
s = pd.Series(data)
s = s.str.strip()
s = s.values.reshape(df.shape)

df = pd.DataFrame(s, columns=df.columns)

df = df.fillna(0)

df1 = pd.read_csv(os.path.join(DATASET_PATH, 'Symptom-severity.csv'))
x=df1['Symptom']

dfx=pd.DataFrame()
dfx["Disease"]=df["Disease"]
y=0
dfx[x]=0
for index, row in df.iterrows():
    for symptom in df.columns[1:]:
        if row[symptom] != 0:
            dfx.loc[index, row[symptom]] = 1
dfx = dfx.fillna(0)
dfx[dfx.columns[1:]]=dfx[dfx.columns[1:]].astype('int')
dfx.columns = dfx.columns.str.strip()

symptom_sums = dfx.iloc[:, 1:].sum(axis=0)
symptoms_with_no_values = symptom_sums[symptom_sums == 0].index.tolist()

columns_to_drop = symptoms_with_no_values
dfx = dfx.drop(columns=columns_to_drop)
dfx[dfx.columns[1:]].sum(axis=0).sort_values()
print(dfx.columns.to_list())
y=df['Disease'].unique()

data = dfx.iloc[:,1:].values
labels = dfx['Disease'].values

cp: cannot stat 'kaggle.json': No such file or directory
chmod: cannot access '/root/.kaggle/kaggle.json': No such file or directory
Path to dataset files: /root/.cache/kagglehub/datasets/itachi9604/disease-symptom-description-dataset/versions/2
Using device: cpu
dataset.csv
symptom_precaution.csv
Symptom-severity.csv
symptom_Description.csv
['Disease', 'itching', 'shivering', 'chills', 'acidity', 'vomiting', 'fatigue', 'anxiety', 'restlessness', 'lethargy', 'cough', 'breathlessness', 'sweating', 'dehydration', 'indigestion', 'headache', 'nausea', 'constipation', 'diarrhoea', 'malaise', 'phlegm', 'congestion', 'dizziness', 'cramps', 'bruising', 'obesity', 'unsteadiness', 'depression', 'irritability', 'polyuria', 'coma', 'palpitations', 'blackheads', 'scurring', 'blister', 'skin rash', 'pus filled pimples', 'mood swings', 'weight loss', 'fast heart rate', 'excessive hunger', 'muscle weakness', 'abnormal menstruation', 'muscle wasting', 'patches in throat', 'high fever', 'extra marital c

In [12]:
scaler = StandardScaler()
data_scaled = scaler.fit_transform(data)

x_train, x_test, y_train, y_test = train_test_split(data_scaled, labels, train_size = 0.8,random_state=42)
print(x_train.shape, x_test.shape, y_train.shape, y_test.shape)

from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
y_train = le.fit_transform(y_train)
y_test = le.transform(y_test)

y=le.classes_
y


X_train_tensor = torch.tensor(x_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.long)
X_test_tensor = torch.tensor(x_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.long)

(3936, 131) (984, 131) (3936,) (984,)


In [3]:


# Load Dataset
symptom_severity_path = '/content/Symptom-severity.csv'
dataset_path = '/content/dataset.csv'

# Load the datasets
symptom_severity = pd.read_csv("/content/Symptom-severity.csv")
dataset = pd.read_csv(dataset_path)

# Merge symptom severity into the dataset
symptom_mapping = dict(zip(symptom_severity['Symptom'], symptom_severity['weight']))

# Replace symptoms with their weights in the dataset
symptom_columns = [col for col in dataset.columns if col.startswith('Symptom')]
for col in symptom_columns:
    dataset[col] = dataset[col].map(symptom_mapping).fillna(0)  # Replace symptoms with weights, fill NaN with 0

# Extract features (X) and labels (y)
X = dataset[symptom_columns].values  # Symptom severity weights as features
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(dataset['Disease'])  # Encode diseases as numeric labels

# Normalize the feature values
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# Prepare PyTorch tensors for training
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.long)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.long)


FileNotFoundError: [Errno 2] No such file or directory: '/content/Symptom-severity.csv'

In [13]:

# Define the MLP Model
class MLPModel(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(MLPModel, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, num_classes)

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        return x


In [ ]:

# Define Model Parameters
input_size = X_train.shape[1]
hidden_size = 128
num_classes = len(label_encoder.classes_)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Initialize Model, Loss, and Optimizer
model = MLPModel(input_size, hidden_size, num_classes).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# Train the Model
epochs = 10
for epoch in range(epochs):
    model.train()
    optimizer.zero_grad()
    outputs = model(X_train_tensor.to(device))
    loss = criterion(outputs, y_train_tensor.to(device))
    loss.backward()
    optimizer.step()
    print(f"Epoch [{epoch+1}/{epochs}], Loss: {loss.item():.4f}")


Epoch [1/10], Loss: 3.7255
Epoch [2/10], Loss: 3.7189
Epoch [3/10], Loss: 3.7124
Epoch [4/10], Loss: 3.7060
Epoch [5/10], Loss: 3.6996
Epoch [6/10], Loss: 3.6932
Epoch [7/10], Loss: 3.6869
Epoch [8/10], Loss: 3.6806
Epoch [9/10], Loss: 3.6743
Epoch [10/10], Loss: 3.6680


In [20]:

# Define Model Parameters
input_size = x_train.shape[1]
hidden_size = 128
num_classes = len(le.classes_)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Initialize Model, Loss, and Optimizer
model = MLPModel(input_size, hidden_size, num_classes).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# Train the Model with Validation
epochs = 50
for epoch in range(epochs):
    model.train()
    optimizer.zero_grad()
    outputs = model(X_train_tensor.to(device))
    loss = criterion(outputs, y_train_tensor.to(device))
    loss.backward()
    optimizer.step()
    _, predicted = torch.max(outputs, 1)
    correct = (predicted == y_train_tensor.to(device)).sum().item()
    train_accuracy = correct / y_train_tensor.size(0)
    val_loss = 0.0
    model.eval()
    with torch.no_grad():
        val_outputs = model(X_test_tensor.to(device))
        val_loss = criterion(val_outputs, y_test_tensor.to(device))  # Validation loss
        _, val_predicted = torch.max(val_outputs, 1)
        val_correct = (val_predicted == y_test_tensor.to(device)).sum().item()
        val_accuracy = val_correct / y_test_tensor.size(0)

    print(f"Epoch [{epoch+1}/{epochs}], "
          f"Training Loss: {loss.item():.4f}, Training Accuracy: {train_accuracy * 100:.2f}%, "
          f"Validation Loss: {val_loss.item():.4f}, Validation Accuracy: {val_accuracy * 100:.2f}%")


Epoch [1/50], Training Loss: 3.7418, Training Accuracy: 4.83%, Validation Loss: 3.6447, Validation Accuracy: 8.84%
Epoch [2/50], Training Loss: 3.6471, Training Accuracy: 9.22%, Validation Loss: 3.5492, Validation Accuracy: 10.06%
Epoch [3/50], Training Loss: 3.5533, Training Accuracy: 11.36%, Validation Loss: 3.4551, Validation Accuracy: 20.43%
Epoch [4/50], Training Loss: 3.4608, Training Accuracy: 19.44%, Validation Loss: 3.3617, Validation Accuracy: 30.59%
Epoch [5/50], Training Loss: 3.3691, Training Accuracy: 29.09%, Validation Loss: 3.2687, Validation Accuracy: 39.53%
Epoch [6/50], Training Loss: 3.2779, Training Accuracy: 39.20%, Validation Loss: 3.1760, Validation Accuracy: 49.80%
Epoch [7/50], Training Loss: 3.1871, Training Accuracy: 49.44%, Validation Loss: 3.0832, Validation Accuracy: 58.74%
Epoch [8/50], Training Loss: 3.0963, Training Accuracy: 58.49%, Validation Loss: 2.9909, Validation Accuracy: 72.36%
Epoch [9/50], Training Loss: 3.0059, Training Accuracy: 70.48%, Val

In [24]:
import pickle
model.eval()
with torch.no_grad():
    val_outputs = model(X_test_tensor.to(device))
    _, val_predicted = torch.max(val_outputs, 1)
    val_correct = (val_predicted == y_test_tensor.to(device)).sum().item()
    val_accuracy = val_correct / y_test_tensor.size(0)

print(f"Testing Accuracy: {val_accuracy * 100:.2f}%")

with open('MLPmodel.pkl', 'wb') as f:
    pickle.dump(model, f)

with open('MLPscaler.pkl', 'wb') as f:
    pickle.dump(scaler, f)


Testing Accuracy: 100.00%


In [29]:

# Prediction Function
def predict_disease_mlp(user_input, model, label_encoder, symptom_columns, device, scaler):
    # Create a DataFrame with all symptom columns initialized to 0
    # user_df = pd.DataFrame(data=[[0] * len(symptom_columns)], columns=symptom_columns)
    with open(model, 'rb') as f:
      model = pickle.load(f)


    # Update values based on user input
    # for symptom, value in user_input.items():
    #     if symptom in user_df.columns:
    #         user_df.loc[0, symptom] = value

    # # Scale the input data
    # input_scaled = scaler.transform(user_df.values)

    # # Convert to PyTorch tensor
    # input_tensor = torch.tensor(input_scaled, dtype=torch.float32).to(device)

    # # Perform prediction
    # model.eval()
    # with torch.no_grad():
    #     output = model(input_tensor)
    #     probabilities = torch.softmax(output, dim=1)
    #     _, predicted = torch.max(probabilities, 1)

    # # Decode prediction
    # predicted_label = label_encoder.inverse_transform(predicted.cpu().numpy())
    # prob_dict = {label_encoder.inverse_transform([i])[0]: prob.item() for i, prob in enumerate(probabilities[0].cpu())}


    user_df = pd.DataFrame([user_input], columns=symptom_columns)
    user_df.fillna(0, inplace=True)
    input_scaled = scaler.transform(user_df.values)
    input_tensor = torch.tensor(input_scaled, dtype=torch.float32).to(device)
    model.eval()
    with torch.no_grad():
        output = model(input_tensor)
        probabilities = torch.softmax(output, dim=1)
        _, predicted = torch.max(probabilities, 1)

    predicted_label = label_encoder.inverse_transform(predicted.cpu().numpy())
    prob_dict = {label_encoder.inverse_transform([i])[0]: prob.item() for i, prob in enumerate(probabilities[0].cpu())}
    return predicted_label[0], prob_dict


symptom_columns = ['itching', 'shivering', 'chills', 'acidity', 'vomiting', 'fatigue', 'anxiety', 'restlessness', 'lethargy', 'cough', 'breathlessness', 'sweating', 'dehydration', 'indigestion', 'headache', 'nausea', 'constipation', 'diarrhoea', 'malaise', 'phlegm', 'congestion', 'dizziness', 'cramps', 'bruising', 'obesity', 'unsteadiness', 'depression', 'irritability', 'polyuria', 'coma', 'palpitations', 'blackheads', 'scurring', 'blister', 'skin rash', 'pus filled pimples', 'mood swings', 'weight loss', 'fast heart rate', 'excessive hunger', 'muscle weakness', 'abnormal menstruation', 'muscle wasting', 'patches in throat', 'high fever', 'extra marital contacts', 'yellowish skin', 'loss of appetite', 'abdominal pain', 'yellowing of eyes', 'chest pain', 'loss of balance', 'lack of concentration', 'blurred and distorted vision', 'drying and tingling lips', 'slurred speech', 'stiff neck', 'swelling joints', 'painful walking', 'dark urine', 'yellow urine', 'receiving blood transfusion', 'receiving unsterile injections', 'visual disturbances', 'burning micturition', 'bladder discomfort', 'foul smell of urine', 'continuous feel of urine', 'irregular sugar level', 'increased appetite', 'joint pain', 'skin peeling', 'small dents in nails', 'inflammatory nails', 'swelling of stomach', 'distention of abdomen', 'history of alcohol consumption', 'fluid overload', 'pain during bowel movements', 'pain in anal region', 'bloody stool', 'irritation in anus', 'acute liver failure', 'stomach bleeding', 'back pain', 'weakness in limbs', 'neck pain', 'mucoid sputum', 'mild fever', 'muscle pain', 'family history', 'continuous sneezing', 'watering from eyes', 'rusty sputum', 'weight gain', 'puffy face and eyes', 'enlarged thyroid', 'brittle nails', 'swollen extremeties', 'swollen legs', 'prominent veins on calf', 'stomach pain', 'spinning movements', 'sunken eyes', 'silver like dusting', 'swelled lymph nodes', 'blood in sputum', 'swollen blood vessels', 'toxic look (typhos)', 'belly pain', 'throat irritation', 'redness of eyes', 'sinus pressure', 'runny nose', 'loss of smell', 'passage of gases', 'cold hands and feets', 'weakness of one body side', 'altered sensorium', 'nodal skin eruptions', 'red sore around nose', 'yellow crust ooze', 'ulcers on tongue', 'spotting  urination', 'pain behind the eyes', 'red spots over body', 'internal itching', 'movement stiffness', 'knee pain', 'hip joint pain', 'dischromic  patches']
# Example Input and Prediction
user_input = {
    'itching': 1,
    'skin rash': 1,
    'nodal skin eruptions': 0,
    'continuous sneezing': 1,
    'shivering': 1,
    'chills': 1,
    'joint pain': 1,
    'stomach pain': 1,
}

predicted_disease, probabilities = predict_disease_mlp(user_input, 'MLPmodel.pkl', le, symptom_columns, device, scaler)
print(f"Predicted Disease: {predicted_disease}")
print(f"Probabilities: {probabilities}")


Predicted Disease: Allergy
Probabilities: {'(vertigo) Paroymsal  Positional Vertigo': 0.012681626714766026, 'AIDS': 0.009954303503036499, 'Acne': 0.006384456064552069, 'Alcoholic hepatitis': 0.010267342440783978, 'Allergy': 0.5073794722557068, 'Arthritis': 0.010051459074020386, 'Bronchial Asthma': 0.016070546582341194, 'Cervical spondylosis': 0.006008610595017672, 'Chicken pox': 0.00544655229896307, 'Chronic cholestasis': 0.022082846611738205, 'Common Cold': 0.01673133112490177, 'Dengue': 0.010190230794250965, 'Diabetes': 0.005187821574509144, 'Dimorphic hemmorhoids(piles)': 0.01695755310356617, 'Drug Reaction': 0.08069169521331787, 'Fungal infection': 0.021872632205486298, 'GERD': 0.015807896852493286, 'Gastroenteritis': 0.01044006273150444, 'Heart attack': 0.01492294017225504, 'Hepatitis B': 0.00521079869940877, 'Hepatitis C': 0.010144032537937164, 'Hepatitis D': 0.009640875272452831, 'Hepatitis E': 0.006964046973735094, 'Hypertension': 0.006437552627176046, 'Hyperthyroidism': 0.0050